## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pisco,PE,2020-11-08 22:08:04,-13.70,-76.22,68.00,72,0,25.28,NaN
1,1,Katobu,ID,2020-11-08 22:08:05,-4.94,122.53,76.41,83,92,1.07,NaN
2,2,Constitucion,CL,2020-11-08 22:03:25,-35.33,-72.42,62.60,58,46,12.41,NaN
3,3,Ponta Do Sol,PT,2020-11-08 22:08:05,32.67,-17.10,66.20,59,20,10.29,NaN
4,4,Labuhan,ID,2020-11-08 22:08:05,-6.88,112.21,78.66,78,94,10.36,NaN


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Katobu,ID,2020-11-08 22:08:05,-4.94,122.53,76.41,83,92,1.07,NaN
4,4,Labuhan,ID,2020-11-08 22:08:05,-6.88,112.21,78.66,78,94,10.36,NaN
5,5,Puerto Ayora,EC,2020-11-08 22:08:05,-0.74,-90.35,77.00,69,20,14.99,NaN
12,12,Vaini,TO,2020-11-08 22:08:06,-21.20,-175.20,77.00,78,20,6.93,NaN
14,14,Durango,MX,2020-11-08 22:05:15,24.83,-104.83,76.82,22,0,7.27,NaN
16,16,Georgetown,MY,2020-11-08 22:04:08,5.41,100.34,77.00,94,40,2.24,NaN
17,17,Bengkulu,ID,2020-11-08 22:08:07,-3.80,102.27,75.60,91,100,1.57,NaN
25,25,Tomatlan,MX,2020-11-08 22:08:08,19.93,-105.25,85.59,59,74,9.84,NaN
31,31,Mecca,SA,2020-11-08 22:08:10,21.43,39.83,77.77,70,1,2.84,NaN
41,41,Mangrol,IN,2020-11-08 22:04:32,21.12,70.12,78.10,55,0,3.60,NaN


In [5]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())
print(preferred_cities_df.notnull().sum())

City_ID                800
City                   800
Country                795
Date                   800
Lat                    800
Lng                    800
Max Temp               800
Humidity               800
Cloudiness             800
Wind Speed             800
Current Description    370
dtype: int64
City_ID                800
City                   800
Country                795
Date                   800
Lat                    800
Lng                    800
Max Temp               800
Humidity               800
Cloudiness             800
Wind Speed             800
Current Description    370
dtype: int64


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1386,Morant Bay,JM,80.60,thunderstorm,17.88,-76.41,
1393,Labuhan,ID,86.85,broken clouds,-6.88,112.21,
1396,Moree,AU,78.80,clear sky,-29.47,149.85,
1401,Vaini,TO,80.60,few clouds,-21.20,-175.20,
1405,Georgetown,MY,82.40,moderate rain,5.41,100.34,
1406,Bengkulu,ID,80.10,moderate rain,-3.80,102.27,
1413,Hobart,AU,75.00,clear sky,-42.88,147.33,
1420,Mecca,SA,75.60,clear sky,21.43,39.83,
1428,Lumphat,KH,79.93,overcast clouds,13.49,106.98,
1430,Mangrol,IN,75.92,clear sky,21.12,70.12,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, IndentationError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset = ["Hotel Name"])
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1386,Morant Bay,JM,80.60,thunderstorm,17.88,-76.41,Whispering Bamboo Cove Resort
1393,Labuhan,ID,86.85,broken clouds,-6.88,112.21,bador sepret
1396,Moree,AU,78.80,clear sky,-29.47,149.85,Molika Springs Motel
1401,Vaini,TO,80.60,few clouds,-21.20,-175.20,Keleti Beach Resort
1405,Georgetown,MY,82.40,moderate rain,5.41,100.34,Cititel Penang


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))